In [ ]:
!pip uninstall typing -y
!pip install neptune-client 
!pip install audiomentations==0.11.0

In [ ]:
!rm -rf kaggle-birdclef-2021
!git clone https://github.com/fraank/kaggle-birdclef-2021.git

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%cd kaggle-birdclef-2021/src

## Environment Variables

Set these environment variables if you plan on adding slack notifications at the end of training loops or plan on using neptune for logging


In [ ]:
# %env SLACK_URL=""
# %env NEPTUNE_API_TOKEN=""

In [ ]:
import os
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

TRAIN = pd.read_csv('../../../input/birdclef-2021/train_metadata.csv',)
print('Complete Train Data: %s' % (len(TRAIN)))

TRAIN = TRAIN.query('rating>=4')
train, test = train_test_split(TRAIN, test_size=0.2, stratify=TRAIN['primary_label'])

# print(TRAIN.groupby('primary_label').size())
# print(train.groupby('primary_label').size())
# print(test.groupby('primary_label').size())

print('Train Data with rating >4: %s' % (len(train)))
print('Test Data with rating >4: %s' % (len(test)))

# save CSVs
if not os.path.exists('../../csv'):
    os.mkdir('../../csv')
train.to_csv('../../csv/train.csv')
test.to_csv('../../csv/test.csv')

# TODO accept ogg
# TODO 

In [ ]:
# Generate Labels for Config
LABELS = {}
i = 0
for item in sorted(TRAIN.primary_label.unique()):
    LABELS[item] = i
    i += 1
print(LABELS)

In [ ]:
# Generate Names for Config
NAMES = {}
label_name = TRAIN.groupby(['primary_label', 'common_name']).count()
for label, name in label_name.index:
    NAMES[label] = name
print(NAMES)

In [ ]:
%%writefile config_params/example_config.py
from pathlib import Path
import torch
from config_params.configs import get_dict_value, BIRD_CODE, INV_EBIRD_LABEL
import os

class Parameters(object):
    def __init__(self, hparams=None):
        self.fold = 0
        self.name = os.path.basename(__file__).replace(".py","")
        
        self.aug_name = "secondary_default"
        self.apply_mixup = False
        self.model_name = "sed_dense121att"

        self.model_config =  {
            "sample_rate": 32000,
            "window_size": 1024,
            "hop_size": 320,
            "mel_bins": 64,
            "fmin": 50,
            "fmax": 14000,
            "classes_num": 397,
            "apply_aug": True,
            "top_db": None
        }
        self.pretrained_path = None

        self.bckgrd_aug_dir = "../../../input/pinknoise" 
        self.secondary_bckgrd_aug_dir = "../../../input/pinknoise"

        self.optimizer_name = "adamw"
        self.weight_decay = 0.01

        self.criterion_name = "sed_scaled_pos_neg_focal_loss"
        self.criterion_params = {
            "gamma" : 0.0,
            "alpha_0" : 1.0,
            "alpha_1": 1.0,
            "secondary_factor": 1.0
        }
        

        self.scheduler_name = "warmup_with_cosine"
        self.lr_scale_factor = 0.01
        self.lr = 0.001

        self.logger_name = "print" # set to "neptune" if using neptune logger instead
        
        self.PERIOD = 30

        self.train_ds_params = {
            "root_dir": get_dict_value(Path("../../../input/birdclef-2021/train_short_audio/")), # Set to Path("data/") if not using Kaggle Kernel if the data is in data/
            "csv_dir": Path(f"../../csv/train.csv"),
            "period": self.PERIOD,
            "bird_code": BIRD_CODE,
            "inv_ebird_label":INV_EBIRD_LABEL,
            "isTraining": True,
            "num_test_samples": 1,
        }
        
        self.valid_ds_params = {
            "root_dir": get_dict_value(Path("../../../input/birdclef-2021/train_short_audio/")), # Set to Path("data/") if not using Kaggle Kernel if the data is in data/
            "csv_dir": Path(f"../../csv/test.csv"),
            "period": self.PERIOD,
            "bird_code": BIRD_CODE,
            "inv_ebird_label":INV_EBIRD_LABEL,
            "background_audio_dir": None,
            "isTraining": False,
            "num_test_samples": 2,
        }

        self.test_ds_params = {
            "root_dir": get_dict_value(Path("../../../input/birdclef-2021/train_short_audio/")), # Set to Path("data/") if not using Kaggle Kernel if the data is in data/
             "csv_dir": Path(f"../../csv/test.csv"),
            "background_audio_dir":  None,
            "period": self.PERIOD,
            "bird_code": BIRD_CODE,
            "inv_ebird_label":INV_EBIRD_LABEL,
            "isTraining": False,
            "num_test_samples": 2,
        }

        self.checkpoint_params = {
            "save_dir":f"../../saved_models/{self.name}", # Path to save the checkpoints
            "n_saved":2,
            "prefix_name":self.name,
        }
        
        self.train_bs = 28
        self.train_num_workers = 2 # changed to 2 workers
        self.valid_bs = 32
        self.valid_num_workers = 1 # changed to 1 workers
        self.metrics = ["lraps", "f1score_clip", "f1score_frame"]
        
        self.track_metric = "f1score_clip"
        self.metric_factor = 1
        
        self.checkpoint_dir = None
        self.add_pbar = True
        
        self.run_params = {
            "max_epochs": 20, # 50,
            "epoch_length": None
        }
        
        self.logger_params = {
            "project_name": "bird-song",
            "log_every": 1, # 10,
            "name": self.name,
            "prefix_name": f"{self.name}_best",
            "tags": [self.fold, self.name, self.model_name, self.criterion_name],
            "params": {
                "bs": self.train_bs,
                "lr": self.lr,
                "name": self.name,
                "aug_name": self.aug_name,
                "model_name": self.model_name,
                "weight_decay": self.weight_decay,
                "apply_mixup": self.apply_mixup,
                "optimizer_name": self.optimizer_name,
                "criterion_name": self.criterion_name,
                "scheduler_name": self.scheduler_name,
                "fold": self.fold,
                "lr_scale_factor": self.lr_scale_factor,
                "period": self.PERIOD,
                **self.model_config,
                **self.criterion_params
            }
        }
        
        self.dist_params = {
        }

        self.val_length = None 
        self.eval_every = 2 # Validates every 2 epochs
        self.load_model_only = True
        self.accumulation_steps = 1
        self.gradient_clip_val = 0

In [ ]:
!python sed_train.py --config "config_params.example_config"

In [ ]:
!ls ../../saved_models/example_config